In [322]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [323]:
# Load the data
sends_df= pd.read_csv("sends.csv", date_parser=True)
users_df = pd.read_csv("User info.csv")
events_df= pd.read_csv("events.txt", sep='\t')

In [324]:
# Change the datetime format and merge the Sends and Users table with calculated tenure
sends_df['send_dt'] = pd.to_datetime(sends_df['send_dt'])
users_df['aq_dt'] = pd.to_datetime(users_df['aq_dt'])
events_df['event_captured_dt']= pd.to_datetime(events_df['event_captured_dt'])

merge1_df= pd.merge(sends_df,users_df,on=['riid'])
merge1_df['tenure']= (merge1_df['send_dt']-merge1_df['aq_dt']).dt.days

In [327]:
#merge1_df.head()

In [328]:
# Unstack the Events table
unstack_df = events_df.reset_index().groupby(['user_id','event_captured_dt', 'event'])['cnt'].aggregate('first').unstack().fillna(0)
unstack_df.reset_index(inplace=True) 
unstack_df.head()

event,user_id,event_captured_dt,Click,Online Purch,Open,Sent
0,82692742,2015-07-02,0.0,0.0,0.0,1.0
1,82692742,2015-07-03,0.0,0.0,0.0,1.0
2,82692742,2015-07-04,0.0,0.0,0.0,1.0
3,82692742,2015-07-07,0.0,0.0,0.0,1.0
4,82692742,2015-07-08,0.0,0.0,0.0,1.0


In [329]:
# Merge the Events and Sends table and calculate the user activity days 
interim_df= pd.merge(sends_df,unstack_df,how='right', left_on=['riid'], right_on=['user_id'])

interim_df['usr_act_dt']=(interim_df['send_dt']-interim_df['event_captured_dt']).dt.days

In [330]:
interim_df.head()

,riid,launch_id,opened,send_dt,user_id,event_captured_dt,Click,Online Purch,Open,Sent,usr_act_dt
0,605591962,784382,0,2017-07-14,605591962,2015-12-02,0.0,0.0,0.0,1.0,590
1,605591962,784382,0,2017-07-14,605591962,2015-12-05,0.0,0.0,0.0,1.0,587
2,605591962,784382,0,2017-07-14,605591962,2016-03-01,0.0,0.0,0.0,1.0,500
3,605591962,784382,0,2017-07-14,605591962,2016-03-03,0.0,0.0,0.0,1.0,498
4,605591962,784382,0,2017-07-14,605591962,2016-03-09,0.0,0.0,0.0,1.0,492


In [331]:
# Captured the user events in the last one week only - May help to understand if the user will open new email or not?
interim_df= interim_df[(interim_df['usr_act_dt']>0) & (interim_df['usr_act_dt']<8)][['riid','launch_id','send_dt','Click','Open','Sent','Online Purch']]
interim_df.head()

,riid,launch_id,send_dt,Click,Open,Sent,Online Purch
231,181990762,422482,2016-07-11,0.0,0.0,1.0,0.0
232,181990762,422482,2016-07-11,0.0,0.0,1.0,0.0
233,181990762,422482,2016-07-11,0.0,0.0,1.0,0.0
234,181990762,422482,2016-07-11,0.0,0.0,1.0,0.0
235,181990762,422482,2016-07-11,0.0,0.0,1.0,0.0


In [332]:
# Sum the user activity in the last one week for each different event

interim_df= interim_df.groupby(['riid','launch_id','send_dt']).sum()
interim_df.reset_index(inplace=True) 
interim_df.head()

,riid,launch_id,send_dt,Click,Open,Sent,Online Purch
0,82697102,784382,2017-07-14,0.0,0.0,4.0,0.0
1,82701822,468302,2016-09-07,0.0,0.0,1.0,0.0
2,82721162,529482,2016-11-25,0.0,1.0,1.0,0.0
3,82723722,503362,2016-10-24,0.0,0.0,4.0,0.0
4,82792482,578522,2017-02-06,0.0,0.0,2.0,0.0


In [333]:
interim_df.shape

(29999, 7)

In [444]:
# Merge the interim table with merge1 to have the final merged table 
merged2_df= pd.merge(merge1_df,interim_df,how='left',on=['riid','launch_id','send_dt'])
merged2_df.head()

,riid,launch_id,opened,send_dt,aq_dt,sub_source,tenure,Click,Open,Sent,Online Purch
0,605591962,784382,0,2017-07-14,2015-10-29,NaN,624,NaN,NaN,NaN,NaN
1,181990762,422482,0,2016-07-11,2014-12-08,NaN,581,0.0,0.0,5.0,0.0
2,88630922,468302,0,2016-09-07,2010-09-26,Retail B&M,2173,NaN,NaN,NaN,NaN
3,259205742,529502,0,2016-11-25,2015-04-14,Online Purchase,591,0.0,0.0,6.0,0.0
4,172633782,558762,0,2017-01-17,2014-06-19,Retail B&M,943,0.0,0.0,2.0,0.0


In [445]:
merged2_df.isnull().any()

riid            False
launch_id       False
opened          False
send_dt         False
aq_dt           False
sub_source       True
tenure          False
Click            True
Open             True
Sent             True
Online Purch     True
dtype: bool

In [446]:
## Feature Transformation

merged2_df['sub_source']= merged2_df['sub_source'].fillna('Unknown')
merged2_df= merged2_df.fillna(0)

merged2_df['send_year']= merged2_df['send_dt'].dt.year
merged2_df['send_month']= merged2_df['send_dt'].dt.month
merged2_df['send_day']= merged2_df['send_dt'].dt.day
#merged2_df.drop(['riid','send_dt','aq_dt'], axis=1,inplace = True)
#merged2_df = pd.get_dummies(data= merged2_df, columns=['launch_id','sub_source','send_year'])

## Checkpoint 

In [447]:
data_df= merged2_df
data_df.shape

(40000, 14)

In [347]:
#data_df.describe().T

In [380]:
# To check the balance of the target label
data_df['opened'].value_counts()

0    33882
1     6118
Name: opened, dtype: int64

In [381]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm


from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import RandomOverSampler

In [382]:
target = data_df['opened']
features= data_df.drop('opened', axis=1)

train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=0.3, random_state=42)

X_train =train_X.drop(['riid','send_dt','aq_dt'], axis=1)
X_train = pd.get_dummies(data= X_train, columns=['launch_id','sub_source','send_year'])


X_test =test_X.drop(['riid','send_dt','aq_dt'], axis=1)
X_test = pd.get_dummies(data= X_test, columns=['launch_id','sub_source','send_year'])



In [383]:
%%time
clf_rf = RandomForestClassifier( random_state=40)
clf_rf.fit(X_train,y_train)

pred= clf_rf.predict(X_test)


print (f1_score(y_test,pred, average='micro'))
print (roc_auc_score(y_test,pred, average='micro'))
print (confusion_matrix(y_test,pred))

0.829083333333
0.616088303113
[[9381  781]
 [1270  568]]
CPU times: user 436 ms, sys: 26.1 ms, total: 463 ms
Wall time: 467 ms


In [384]:
%%time
clf_lr = LogisticRegression( random_state=40)
clf_lr.fit(X_train,y_train)

pred= clf_lr.predict(X_test)


print (f1_score(y_test,pred, average='micro'))
print (roc_auc_score(y_test,pred, average='micro'))
print (confusion_matrix(y_test,pred))

0.862416666667
0.591203086709
[[9981  181]
 [1470  368]]
CPU times: user 238 ms, sys: 19.8 ms, total: 257 ms
Wall time: 263 ms


In [385]:
%%time
clf_svm = svm.SVC( random_state=40)
clf_svm.fit(X_train,y_train)

pred= clf_svm.predict(X_test)


print (f1_score(y_test,pred, average='micro'))
print (roc_auc_score(y_test,pred, average='micro'))
print (confusion_matrix(y_test,pred))

0.8495
0.527645826404
[[10077    85]
 [ 1721   117]]
CPU times: user 2min 47s, sys: 1.83 s, total: 2min 49s
Wall time: 2min 52s


## Normalization with Oversampling

In [448]:
data_df= merged2_df
target= data_df['opened']
features= data_df.drop('opened',axis=1)
#features= (features-features.mean())/features.std() # Standardization

train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=0.3, random_state=42)

X_train =train_X.drop(['riid','send_dt','aq_dt'], axis=1)
X_train = pd.get_dummies(data= X_train, columns=['launch_id','sub_source','send_year'])
X_train = (X_train - X_train.mean())/X_train.std() # Standardization

X_test =test_X.drop(['riid','send_dt','aq_dt'], axis=1)
X_test = pd.get_dummies(data= X_test, columns=['launch_id','sub_source','send_year'])
X_test = (X_test - X_test.mean())/X_test.std() # Standardization

In [449]:
#  Oversampling 
rus = RandomOverSampler(random_state=42)
X_res, y_res = rus.fit_sample(X_train,y_train)
print (X_res.shape)
print (y_res.shape)
print (np.bincount(y_res))

(47440, 58)
(47440,)
[23720 23720]


In [433]:
%%time
clf_rf = RandomForestClassifier( random_state=40)
clf_rf.fit(X_res,y_res)

pred= clf_rf.predict(X_test)


print (f1_score(y_test,pred, average='micro'))
print (roc_auc_score(y_test,pred, average='micro'))
print (confusion_matrix(y_test,pred))

0.803083333333
0.634161780462
[[8919 1243]
 [1120  718]]
CPU times: user 576 ms, sys: 29.4 ms, total: 605 ms
Wall time: 619 ms


In [439]:
%%time
clf_svm = svm.SVC( random_state=40)
clf_svm.fit(X_res,y_res)

pred= clf_svm.predict(X_test)


print (f1_score(y_test,pred, average='micro'))
print (roc_auc_score(y_test,pred, average='micro'))
print (confusion_matrix(y_test,pred))

0.806833333333
0.710133272969
[[8633 1529]
 [ 789 1049]]
CPU times: user 4min 54s, sys: 2.66 s, total: 4min 57s
Wall time: 5min


In [453]:
%%time
clf_lr = LogisticRegression(max_iter=1000, random_state=40)
clf_lr.fit(X_res,y_res)

pred= clf_lr.predict(X_test)

print (f1_score(y_test,pred, average='micro'))
print (roc_auc_score(y_test,pred, average='micro'))
print (confusion_matrix(y_test,pred))

0.820916666667
0.715328918527
[[8816 1346]
 [ 803 1035]]
CPU times: user 383 ms, sys: 14.7 ms, total: 398 ms
Wall time: 404 ms


In [454]:
clf_lr.classes_

array([0, 1])

## Information for Marketing Manager

In [459]:
# Predicting the prob of class 1
prob_opened= clf_lr.predict_proba(X_test)[:,1]

test_result= test_X[['riid','launch_id','send_dt']]
test_result['prob_opened']=prob_opened
test_result.head()

/Users/rajegupt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,riid,launch_id,send_dt,prob_opened
32823,829467242,784382,2017-07-14,0.376369
16298,705582442,684802,2017-05-08,0.407312
28505,97476602,525622,2016-11-22,0.675860
6689,105140942,529502,2016-11-25,0.781448
26893,119748782,784382,2017-07-14,0.322868


In [460]:
test_X['riid']

32823    829467242
16298    705582442
28505     97476602
6689     105140942
26893    119748782
36572    582643262
12335    650437102
29591    177081722
18948    525888082
31067    177712462
22241    676767122
13787    519169122
5128     136856442
9125     543219362
35041    689852622
3510     714303082
16441    519744242
19463    145075782
26283    157458702
36216    135463102
17407    734266902
10919    169149422
30866    708206822
34436    566628242
12701    643312942
35676    747705762
34089    176617742
21261    637541442
26173    656448122
28283    677897322
           ...    
10623    583934362
11414    635335102
351      621546082
9489     561153622
36982    522534002
14839    677405822
33241    535698942
24766    683193222
35343    624327502
36005    710081482
6373     650973942
35008    106858722
38436    418226322
11469    104869842
6713     139044362
35411    712146402
17702    712475262
3475      97629962
37345    529859042
6280     588434682
5953     574792182
17224    129

In [461]:
# Important KPIs
imp_kpi= pd.DataFrame(np.array(X_train.columns))
imp_kpi['weighted_impt']=pd.DataFrame((clf_lr.coef_).T)
imp_kpi

,0,weighted_impt
0,tenure,0.051612
1,Click,-0.060230
2,Open,1.579091
3,Sent,-0.136556
4,Online Purch,-0.009044
5,send_month,0.003658
6,send_day,0.029020
7,launch_id_422482,-0.013568
8,launch_id_435222,0.003279
9,launch_id_446722,0.024122


In [462]:
test_result.to_csv("Q4_Test_result_pred_prob.csv")
imp_kpi.to_csv("imp_kpi_log_reg.csv")